![Vespa logo](https://vespa.ai/assets/vespa-logo-color.png)

# LightGBM: Training the model with Vespa features

The main goal of this tutorial is to deploy and use a LightGBM model in a Vespa application. The following tasks will be accomplished throughout the tutorial:

1. Train a LightGBM classification model with variable names supported by Vespa. 
2. Create Vespa application package files and export then to an application folder.
3. Export the trained LightGBM model to the Vespa application folder.
4. Deploy the Vespa application using the application folder.
5. Feed data to the Vespa application.
6. Assert that the LightGBM predictions from the deployed model are correct.

<div class="alert alert-info">
    Refer to <a href="https://pyvespa.readthedocs.io/en/latest/troubleshooting.html">troubleshooting</a>
    for any problem when running this guide.
</div>

## Setup

Install and load required packages. 

In [ ]:
!pip install numpy pandas pyvespa lightgbm

In [ ]:
import json
import lightgbm as lgb
import numpy as np
import pandas as pd

## Create data

Generate a toy dataset to follow along. Note that we set the column names in a format that Vespa understands. `query(value)` means that the user will send a parameter named `value` along with the query. `attribute(field)` means that `field` is a document attribute defined in a schema. In the example below we have a query parameter named `value` and two document's attributes, `numeric` and `categorical`. If we want `lightgbm` to handle categorical variables we should use `dtype="category"` when creating the dataframe, as shown below.

In [ ]:
# Create random training set
features = pd.DataFrame({
            "query(value)": np.random.random(100),
            "attribute(numeric)": np.random.random(100),
            "attribute(categorical)": pd.Series(np.random.choice(["a", "b", "c"], size=100), dtype="category")
        })
features.head()

,query(value),attribute(numeric),attribute(categorical)
0,0.554113,0.998608,a
1,0.224981,0.666061,b
2,0.860146,0.659856,c
3,0.497353,0.555479,a
4,0.240207,0.427420,c


We generate the target variable as a function of the three features defined above:

In [ ]:
numeric_features = pd.get_dummies(features)
targets = (
    (numeric_features["query(value)"] + 
     numeric_features["attribute(numeric)"]  -
     0.5 * numeric_features["attribute(categorical)_a"] + 
     0.5 * numeric_features["attribute(categorical)_c"]) > 1.0
) * 1.0
targets

0     1.0
1     0.0
2     1.0
3     0.0
4     1.0
     ... 
95    0.0
96    0.0
97    0.0
98    0.0
99    1.0
Length: 100, dtype: float64

## Fit lightgbm model

Train an LightGBM model with a binary loss function:

In [ ]:
training_set = lgb.Dataset(features, targets)

# Train the model
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 3,
}
model = lgb.train(params, training_set, num_boost_round=5)

[LightGBM] [Info] Number of positive: 47, number of negative: 53
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000943 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 74
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470000 -> initscore=-0.120144
[LightGBM] [Info] Start training from score -0.120144


## Vespa application package

Create a Vespa application package. The model expects two document attributes, `numeric` and `categorical`. We can use the model in the first-phase ranking by using the `lightgbm` rank feature.

In [ ]:
from vespa.package import ApplicationPackage, Field, RankProfile, Function

app_package = ApplicationPackage(name="lightgbm")
app_package.schema.add_fields(
    Field(name="id", type="string", indexing=["summary", "attribute"]),
    Field(name="numeric", type="double", indexing=["summary", "attribute"]),
    Field(name="categorical", type="string", indexing=["summary", "attribute"])
)
app_package.schema.add_rank_profile(
    RankProfile(
        name="classify", 
        first_phase="lightgbm('lightgbm_model.json')"
    )
)

We can check how the Vespa search defition file will look like:

In [ ]:
print(app_package.schema.schema_to_text)

schema lightgbm {
    document lightgbm {
        field id type string {
            indexing: summary | attribute
        }
        field numeric type double {
            indexing: summary | attribute
        }
        field categorical type string {
            indexing: summary | attribute
        }
    }
    rank-profile classify {
        first-phase {
            expression: lightgbm('lightgbm_model.json')
        }
    }
}


We can export the application package files to disk:

In [ ]:
from pathlib import Path
Path("lightgbm").mkdir(parents=True, exist_ok=True)
app_package.to_files("lightgbm")

Note that we don't have any models under the `models` folder. We need to export the lightGBM model that we trained earlier to `models/lightgbm.json`.

In [ ]:
!tree lightgbm

lightgbm
├── files
├── models
├── schemas
│   └── lightgbm.sd
├── search
│   └── query-profiles
│       ├── default.xml
│       └── types
│           └── root.xml
└── services.xml

6 directories, 4 files


## Export the model

In [ ]:
with open("lightgbm/models/lightgbm_model.json", "w") as f:
    json.dump(model.dump_model(), f, indent=2)

Now we can see that the model is where Vespa expects it to be:

In [ ]:
!tree lightgbm

lightgbm
├── files
├── models
│   └── lightgbm_model.json
├── schemas
│   └── lightgbm.sd
├── search
│   └── query-profiles
│       ├── default.xml
│       └── types
│           └── root.xml
└── services.xml

6 directories, 5 files


## Deploy the application

Deploy the application package from disk with Docker:

In [ ]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy_from_disk(application_name="lightgbm", application_root="lightgbm")

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Waiting for configuration server, 10/300 seconds...
Waiting for application status, 0/300 seconds...
Waiting for application status, 5/300 seconds...
Waiting for application status, 10/300 seconds...
Waiting for application status, 15/300 seconds...
Waiting for application status, 20/300 seconds...
Waiting for application status, 25/300 seconds...
Waiting for application status, 30/300 seconds...
Finished deployment.


## Feed the data

Feed the simulated data. To feed data in batch we need to create a list of dictionaries containing `id` and `fields` keys:

In [ ]:
feed_batch = [
    {
        "id": idx, 
        "fields": {
            "id": idx,
            "numeric": row["attribute(numeric)"], 
            "categorical": row["attribute(categorical)"]
        }
    } for idx, row in features.iterrows()
]

Feed the batch of data:

In [ ]:
from vespa.io import VespaResponse 

def callback(response:VespaResponse, id:str):
    if not response.is_successfull():
        print(f"Document {id} was not fed to Vespa due to error: {response.get_json()}")
   
app.feed_iterable(feed_batch, callback=callback)

Successful documents fed: 100/100.
Batch progress: 1/1.


## Model predictions

Predict with the trained LightGBM model so that we can later compare with the predictions returned by Vespa.

In [ ]:
features["model_prediction"] = model.predict(features)

In [ ]:
features

,query(value),attribute(numeric),attribute(categorical),model_prediction
0,0.554113,0.998608,a,0.455583
1,0.224981,0.666061,b,0.302412
2,0.860146,0.659856,c,0.626416
3,0.497353,0.555479,a,0.302412
4,0.240207,0.427420,c,0.626416
...,...,...,...,...
95,0.077941,0.686131,b,0.302412
96,0.634318,0.125230,b,0.455583
97,0.116513,0.477102,b,0.302412
98,0.600862,0.872790,a,0.455583


## Query

Create a `compute_vespa_relevance` function that takes a document `id` and a query `value` and return the LightGBM model deployed.

In [ ]:
def compute_vespa_relevance(id_value, query_value):
    hits = app.query(
        body={
            "yql": "select * from sources * where id = {}".format(str(id_value)),
            "ranking": "classify",
            "ranking.features.query(value)": features.loc[id_value, "query(value)"],
            "hits": 1
        }
    ).hits
    return hits[0]["relevance"]

compute_vespa_relevance(id_value=0, query_value = 0.655225)

0.4555830422953402

Loop through the `features` to compute a vespa prediction for all the data points, so that we can compare it to the predictions made by the model outside Vespa.

In [ ]:
vespa_relevance = []
for idx, row in features.iterrows():
    vespa_relevance.append(compute_vespa_relevance(id_value=idx, query_value = row["query(value)"]))
features["vespa_relevance"] = vespa_relevance

In [ ]:
features

,query(value),attribute(numeric),attribute(categorical),model_prediction,vespa_relevance
0,0.554113,0.998608,a,0.455583,0.455583
1,0.224981,0.666061,b,0.302412,0.302412
2,0.860146,0.659856,c,0.626416,0.626416
3,0.497353,0.555479,a,0.302412,0.302412
4,0.240207,0.427420,c,0.626416,0.626416
...,...,...,...,...,...
95,0.077941,0.686131,b,0.302412,0.302412
96,0.634318,0.125230,b,0.455583,0.455583
97,0.116513,0.477102,b,0.302412,0.302412
98,0.600862,0.872790,a,0.455583,0.455583


## Compare model and Vespa predictions

Predictions from the model should be equal to predictions from Vespa, showing the model was correctly deployed to Vespa.

In [ ]:
assert features["model_prediction"].tolist() == features["vespa_relevance"].tolist()

## Clean environment

In [ ]:
!rm -fr lightgbm
vespa_docker.container.stop()
vespa_docker.container.remove()